### Importing the Libraries

In [17]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [18]:
tf.__version__ 

'2.12.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [19]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [20]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
training_set = train_datagen.flow_from_directory('./drive/MyDrive/dataSet2/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 7003 images belonging to 27 classes.


In [23]:
test_set = test_datagen.flow_from_directory('./drive/MyDrive/dataSet2/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 1773 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [24]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [25]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [26]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [27]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [28]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [29]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [30]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [31]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [32]:
classifier.fit(training_set,
                  epochs = 10,
                  validation_data = test_set)

Epoch 1/10
701/701 [==============================] - 2557s 4s/step - loss: 2.9000 - accuracy: 0.1068 - val_loss: 2.0656 - val_accuracy: 0.2566
Epoch 2/10
701/701 [==============================] - 191s 272ms/step - loss: 1.9792 - accuracy: 0.3090 - val_loss: 1.2700 - val_accuracy: 0.5042
Epoch 3/10
701/701 [==============================] - 187s 266ms/step - loss: 1.5260 - accuracy: 0.4515 - val_loss: 0.9003 - val_accuracy: 0.6362
Epoch 4/10
701/701 [==============================] - 193s 276ms/step - loss: 1.2178 - accuracy: 0.5653 - val_loss: 0.8410 - val_accuracy: 0.7090
Epoch 5/10
701/701 [==============================] - 192s 274ms/step - loss: 0.9831 - accuracy: 0.6449 - val_loss: 0.6319 - val_accuracy: 0.7800
Epoch 6/10
701/701 [==============================] - 188s 268ms/step - loss: 0.8438 - accuracy: 0.6904 - val_loss: 0.6619 - val_accuracy: 0.7580
Epoch 7/10
701/701 [==============================] - 192s 274ms/step - loss: 0.7309 - accuracy: 0.7358 - val_loss: 0.5744 - v

#### Saving the Model

In [33]:
model_json = classifier.to_json()
with open("model_new_9.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new_9.h5')
print('Weights saved')

Model Saved
Weights saved
